In [1]:
from time import time
import pymssql
import json
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
from sklearn.tree import DecisionTreeRegressor
from time import sleep
import _mssql


# 서버 로그인 정보 json 파일 불러오기
login_data=open('C:/Users/dap/login.json').read()
login = json.loads(login_data)

use_datetime_data=open('C:/Users/dap/use_datetime.json').read()
use_datetime = json.loads(use_datetime_data)

In [2]:
    # 서버 연결
    cnxn = pymssql.connect(server = login['server'], user = login['user'], password = login['password'],database = login['database'])  
    cursor = cnxn.cursor()

    # DB WeatherFactor 불러오기
    cmd_WF = "select * from WeatherFactor order by TimeStamp"
    cursor.execute(cmd_WF)

    A = pd.read_sql(cmd_WF, cnxn)

    # DB YardFactor 불러오기
    cmd_YF = "select * from YardFactor order by TimeStamp"
    cursor.execute(cmd_YF)

    B = pd.read_sql(cmd_YF, cnxn)

    # DB airkorea 및 reference data 불러오기
    cmd_REF = "select * from ReferenceData order by TimeStamp"
    cursor.execute(cmd_REF)

    C = pd.read_sql(cmd_REF, cnxn)

    WF = A[A['TimeStamp'] > use_datetime['start']]
    YF = B[B['TimeStamp'] > use_datetime['start']]
    REF = C[C['TimeStamp'] > use_datetime['start']]

    REF = REF[['TimeStamp', 'PM10Yard', 'PM10AirKorea']].reset_index(drop=True)
    WF = WF.reset_index(drop=True)
    YF = YF.reset_index(drop=True)

    # microseconds 삭제
    WF['TimeStamp'] = WF['TimeStamp'].apply(lambda x: x.replace(microsecond=0))
    YF['TimeStamp'] = YF['TimeStamp'].apply(lambda x: x.replace(microsecond=0))
    REF['TimeStamp'] = REF['TimeStamp'].apply(lambda x: x.replace(microsecond=0))

    data = pd.merge(REF,WF)
    data = pd.merge(data,YF)

    data = data[~(data['Temperature'] > 40)]
    data = data[~(data['Temperature'] < 0)]
    data = data[~(data['WindSpeed'] > 10)]
    data = data[~(data['VisibilityDistance'] < 100)]
    data = data[~(data['PM10AirKorea'] ==0)]
    data = data[~(data['PM10Yard'] == 0)]

    datetime = pd.DatetimeIndex(data['TimeStamp'])
    data['month'] = datetime.month
    data['hour'] = datetime.hour
    data['dayofweek'] = datetime.dayofweek


In [3]:
feature_names= ['PM10AirKorea', 'Temperature',
               'RelativeHumidity', 'WindSpeed', 'WindDirection', 'SolarIrradiation',
               'AtmospherePressure', 'DustCurrent', 'VisibilityDistance',
               'YardVibration', 'SurfaceTemperature', 'month', 'hour', 'dayofweek']

X = data[feature_names]
Y = data['PM10Yard']

X_train = X
Y_train = Y

In [8]:
from keras.layers import LSTM 
from keras.models import Sequential 
from keras.layers import Dense 
import keras.backend as K 
from keras.callbacks import EarlyStopping

In [29]:
K.clear_session()
modela = Sequential()
modela.add(LSTM(34, input_shape=(14,1)))
modela.add(Dense(1))
modela.compile(loss='mean_squared_error', optimizer='adam')
modela.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 34)                4896      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 35        
Total params: 4,931
Trainable params: 4,931
Non-trainable params: 0
_________________________________________________________________


In [30]:
X_train_x = X_train.values
Y_train_x = Y_train.values

X_train_x = X_train_x.reshape(X_train_x.shape[0],14,1)

In [31]:
early_stop = EarlyStopping(monitor='loss', patience=1, verbose=1)

modela.fit(X_train, Y_train, batch_size=30, verbose=1, callbacks=[early_stop])

ValueError: Error when checking input: expected lstm_1_input to have 3 dimensions, but got array with shape (4761, 14)

In [23]:
# Create a based model a
modela = RandomForestRegressor(random_state=105,
                              n_estimators=200,
                              max_features=5,
                              max_depth=30)

# param_grid = { 
#     'n_estimators': [100,200],
#     'max_features': ['auto', 'sqrt'],
#      'max_features': [5,7,10,14],
#     'max_depth' : [10,20,30,40,50]
# }
# from sklearn.model_selection import GridSearchCV
# cv_modela = GridSearchCV(estimator=modela, param_grid=param_grid, cv= 5)
# cv_modela.fit(X_train, Y_train)
# cv_modela.best_params_

modela.fit(X_train, Y_train)

filename = 'C:/Users/dap/model_1.sav'
joblib.dump(modela, filename)

model_a_score = modela.score(X_train, Y_train)
model_a_score = model_a_score *100

modelb = RandomForestRegressor(random_state=105,
                              n_estimators=200,
                              max_features=10,
                              max_depth=30)

modelb.fit(X_train, Y_train)

filename = 'C:/Users/dap/model_2.sav'
joblib.dump(modelb, filename)

model_b_score = modelb.score(X_train, Y_train)
model_b_score = model_b_score *100

modelc = DecisionTreeRegressor(random_state=105,
                              min_samples_leaf=1,
                              max_depth=5)

# param_grid = { 'min_samples_leaf':[1,2,3],
#            'min_samples_split':[0.01,0.05,0.5,2],
#     'max_depth' : [10,20,30,40,50]
# }
# from sklearn.model_selection import GridSearchCV
# cv_modelc = GridSearchCV(estimator=modelc, param_grid=param_grid, cv= 5)
# cv_modelc.fit(X_train, Y_train)
# cv_modelc.best_params_

modelc.fit(X_train, Y_train)

filename = 'C:/Users/dap/model_3.sav'
joblib.dump(modelc, filename)

model_c_score = modelc.score(X_train, Y_train)
model_c_score = model_c_score *100

In [163]:
def Modeling():
    
    # 서버 연결
    cnxn = pymssql.connect(server = login['server'], user = login['user'], password = login['password'],database = login['database'])  
    cursor = cnxn.cursor()

    # DB WeatherFactor 불러오기
    cmd_WF = "select * from WeatherFactor order by TimeStamp"
    cursor.execute(cmd_WF)

    A = pd.read_sql(cmd_WF, cnxn)

    # DB YardFactor 불러오기
    cmd_YF = "select * from YardFactor order by TimeStamp"
    cursor.execute(cmd_YF)

    B = pd.read_sql(cmd_YF, cnxn)

    # DB airkorea 및 reference data 불러오기
    cmd_REF = "select * from ReferenceData order by TimeStamp"
    cursor.execute(cmd_REF)

    C = pd.read_sql(cmd_REF, cnxn)

    WF = A[A['TimeStamp'] > use_datetime['start']]
    YF = B[B['TimeStamp'] > use_datetime['start']]
    REF = C[C['TimeStamp'] > use_datetime['start']]

    REF = REF[['TimeStamp', 'PM10Yard', 'PM10AirKorea']].reset_index(drop=True)
    WF = WF.reset_index(drop=True)
    YF = YF.reset_index(drop=True)

    # microseconds 삭제
    WF['TimeStamp'] = WF['TimeStamp'].apply(lambda x: x.replace(microsecond=0))
    YF['TimeStamp'] = YF['TimeStamp'].apply(lambda x: x.replace(microsecond=0))
    REF['TimeStamp'] = REF['TimeStamp'].apply(lambda x: x.replace(microsecond=0))

    data = pd.merge(REF,WF)
    data = pd.merge(data,YF)

    data = data[~(data['Temperature'] > 40)]
    data = data[~(data['Temperature'] < 0)]
    data = data[~(data['WindSpeed'] > 10)]
    data = data[~(data['VisibilityDistance'] < 100)]
    data = data[~(data['PM10AirKorea'] ==0)]
    data = data[~(data['PM10Yard'] == 0)]

    datetime = pd.DatetimeIndex(data['TimeStamp'])
    data['month'] = datetime.month
    data['hour'] = datetime.hour
    data['dayofweek'] = datetime.dayofweek

    feature_names= ['PM10AirKorea', 'Temperature',
               'RelativeHumidity', 'WindSpeed', 'WindDirection', 'SolarIrradiation',
               'AtmospherePressure', 'DustCurrent', 'VisibilityDistance',
               'YardVibration', 'SurfaceTemperature', 'month', 'hour', 'dayofweek']

    X = data[feature_names]
    Y = data['PM10Yard']

    X_train = X
    Y_train = Y

   # Create a based model a
    modela = RandomForestRegressor(random_state=105,
                                  n_estimators=200,
                                  max_features=5,
                                  max_depth=30)

    # param_grid = { 
    #     'n_estimators': [100,200],
    #     'max_features': ['auto', 'sqrt'],
    #      'max_features': [5,7,10,14],
    #     'max_depth' : [10,20,30,40,50]
    # }
    # from sklearn.model_selection import GridSearchCV
    # cv_modela = GridSearchCV(estimator=modela, param_grid=param_grid, cv= 5)
    # cv_modela.fit(X_train, Y_train)
    # cv_modela.best_params_

    modela.fit(X_train, Y_train)

    filename = 'C:/Users/dap/model_1.sav'
    joblib.dump(modela, filename)

    model_a_score = modela.score(X_train, Y_train)
    model_a_score = model_a_score *100

    modelb = RandomForestRegressor(random_state=105,
                                  n_estimators=200,
                                  max_features=10,
                                  max_depth=30)

    modelb.fit(X_train, Y_train)

    filename = 'C:/Users/dap/model_2.sav'
    joblib.dump(modelb, filename)

    model_b_score = modelb.score(X_train, Y_train)
    model_b_score = model_b_score *100

    modelc = DecisionTreeRegressor(random_state=105,
                                  min_samples_leaf=1,
                                  max_depth=5)

    # param_grid = { 'min_samples_leaf':[1,2,3],
    #            'min_samples_split':[0.01,0.05,0.5,2],
    #     'max_depth' : [10,20,30,40,50]
    # }
    # from sklearn.model_selection import GridSearchCV
    # cv_modelc = GridSearchCV(estimator=modelc, param_grid=param_grid, cv= 5)
    # cv_modelc.fit(X_train, Y_train)
    # cv_modelc.best_params_

    modelc.fit(X_train, Y_train)

    filename = 'C:/Users/dap/model_3.sav'
    joblib.dump(modelc, filename)

    model_c_score = modelc.score(X_train, Y_train)
    model_c_score = model_c_score *100

    from datetime import datetime
    timestamp = datetime.now()

    cmd = "update ModelScore set TimeStamp = %s"

    cursor.execute(cmd, timestamp)
    cnxn.commit() 

    modela_score = float("{:.2f}".format(model_a_score))

    cmd = "update ModelScore set ModelA_score = %s"

    cursor.execute(cmd, modela_score)
    cnxn.commit() 

    modelb_score = float("{:.2f}".format(model_b_score))

    cmd = "update ModelScore set ModelB_score = %s"

    cursor.execute(cmd, modelb_score)
    cnxn.commit()         

    modelc_score = float("{:.2f}".format(model_c_score))

    cmd = "update ModelScore set ModelC_score = %s"

    cursor.execute(cmd, modelc_score)
    cnxn.commit()

In [67]:
from time import time
import pymssql
import json
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib
from tkinter import *
from tkinter import ttk
from tkinter import messagebox
from sklearn.tree import DecisionTreeRegressor
from time import sleep
import _mssql


# 서버 로그인 정보 json 파일 불러오기
login_data=open('C:/Users/dap/login.json').read()
login = json.loads(login_data)

use_datetime_data=open('C:/Users/dap/use_datetime.json').read()
use_datetime = json.loads(use_datetime_data)

In [68]:
def Modeling():
    
    # 서버 연결
    cnxn = pymssql.connect(server = login['server'], user = login['user'], password = login['password'],database = login['database'])  
    cursor = cnxn.cursor()

    # DB WeatherFactor 불러오기
    cmd_WF = "select * from WeatherFactor order by TimeStamp"
    cursor.execute(cmd_WF)

    A = pd.read_sql(cmd_WF, cnxn)

    # DB YardFactor 불러오기
    cmd_YF = "select * from YardFactor order by TimeStamp"
    cursor.execute(cmd_YF)

    B = pd.read_sql(cmd_YF, cnxn)

    # DB airkorea 및 reference data 불러오기
    cmd_REF = "select * from ReferenceData order by TimeStamp"
    cursor.execute(cmd_REF)

    C = pd.read_sql(cmd_REF, cnxn)

    WF = A[A['TimeStamp'] > use_datetime['start']]
    YF = B[B['TimeStamp'] > use_datetime['start']]
    REF = C[C['TimeStamp'] > use_datetime['start']]

    REF = REF[['TimeStamp', 'PM10Yard', 'PM10AirKorea']].reset_index(drop=True)
    WF = WF.reset_index(drop=True)
    YF = YF.reset_index(drop=True)

    # microseconds 삭제
    WF['TimeStamp'] = WF['TimeStamp'].apply(lambda x: x.replace(microsecond=0))
    YF['TimeStamp'] = YF['TimeStamp'].apply(lambda x: x.replace(microsecond=0))
    REF['TimeStamp'] = REF['TimeStamp'].apply(lambda x: x.replace(microsecond=0))

    data = pd.merge(REF,WF)
    data = pd.merge(data,YF)

    data = data[~(data['Temperature'] > 40)]
    data = data[~(data['Temperature'] < 0)]
    data = data[~(data['WindSpeed'] > 10)]
    data = data[~(data['VisibilityDistance'] < 100)]
    data = data[~(data['PM10AirKorea'] ==0)]
    data = data[~(data['PM10Yard'] == 0)]

    datetime = pd.DatetimeIndex(data['TimeStamp'])
    data['month'] = datetime.month
    data['hour'] = datetime.hour
    data['dayofweek'] = datetime.dayofweek

    feature_names= ['PM10AirKorea', 'Temperature',
               'RelativeHumidity', 'WindSpeed', 'WindDirection', 'SolarIrradiation',
               'AtmospherePressure', 'DustCurrent', 'VisibilityDistance',
               'YardVibration', 'SurfaceTemperature', 'month', 'hour', 'dayofweek']

    X = data[feature_names]
    Y = data['PM10Yard']

    X_train = X
    Y_train = Y

   # Create a based model a
    modela = RandomForestRegressor(random_state=105,
                                  n_estimators=200,
                                  max_features=5,
                                  max_depth=30)

    # param_grid = { 
    #     'n_estimators': [100,200],
    #     'max_features': ['auto', 'sqrt'],
    #      'max_features': [5,7,10,14],
    #     'max_depth' : [10,20,30,40,50]
    # }
    # from sklearn.model_selection import GridSearchCV
    # cv_modela = GridSearchCV(estimator=modela, param_grid=param_grid, cv= 5)
    # cv_modela.fit(X_train, Y_train)
    # cv_modela.best_params_

    modela.fit(X_train, Y_train)

    filename = 'C:/Users/dap/model_1.sav'
    joblib.dump(modela, filename)

    model_a_score = modela.score(X_train, Y_train)
    model_a_score = model_a_score *100

    modelb = RandomForestRegressor(random_state=105,
                                  n_estimators=200,
                                  max_features=10,
                                  max_depth=30)

    modelb.fit(X_train, Y_train)

    filename = 'C:/Users/dap/model_2.sav'
    joblib.dump(modelb, filename)

    model_b_score = modelb.score(X_train, Y_train)
    model_b_score = model_b_score *100

    modelc = DecisionTreeRegressor(random_state=105,
                                  min_samples_leaf=1,
                                  max_depth=5)

    # param_grid = { 'min_samples_leaf':[1,2,3],
    #            'min_samples_split':[0.01,0.05,0.5,2],
    #     'max_depth' : [10,20,30,40,50]
    # }
    # from sklearn.model_selection import GridSearchCV
    # cv_modelc = GridSearchCV(estimator=modelc, param_grid=param_grid, cv= 5)
    # cv_modelc.fit(X_train, Y_train)
    # cv_modelc.best_params_

    modelc.fit(X_train, Y_train)

    filename = 'C:/Users/dap/model_3.sav'
    joblib.dump(modelc, filename)

    model_c_score = modelc.score(X_train, Y_train)
    model_c_score = model_c_score *100

    from datetime import datetime
    timestamp = datetime.now()

    cmd = "update ModelScore set TimeStamp = %s"

    cursor.execute(cmd, timestamp)
    cnxn.commit() 

    modela_score = float("{:.2f}".format(model_a_score))

    cmd = "update ModelScore set ModelA_score = %s"

    cursor.execute(cmd, modela_score)
    cnxn.commit() 

    modelb_score = float("{:.2f}".format(model_b_score))

    cmd = "update ModelScore set ModelB_score = %s"

    cursor.execute(cmd, modelb_score)
    cnxn.commit()         

    modelc_score = float("{:.2f}".format(model_c_score))

    cmd = "update ModelScore set ModelC_score = %s"

    cursor.execute(cmd, modelc_score)
    cnxn.commit()

In [69]:
from tkinter import *
from tkinter import ttk
import time
import threading
from tkinter import messagebox

try: import tkinter
except ImportError:
    import Tkinter as tkinter
    import ttk

else: 
    from tkinter import ttk

app = Tk()
app.title("AI Dust Prediction Modeling Program")
app.minsize(1035,376)
app.maxsize(1035,376)
frame = Canvas(app, height=376, width=1035)
image1 = PhotoImage(file='C:/Users/dap/gui.png')
frame.create_image(1035,0 , anchor = NE, image=image1)
frame.pack(side=TOP)

def finish_prog():
    if messagebox.askokcancel("Quit", "사용을 종료하시겠습니까?"):
        app.destroy()
    
def start_thread():
    button1['state'] = 'disable'
    progress_bar.start()
    secondary_thread.start()
    app.after(100, check_thread)

def check_thread():
    if secondary_thread.is_alive():
        app.after(100, check_thread)
    else:
        progress_bar.stop()
        button1['state'] ='normal'
        messagebox.showinfo('Completed', '예측 모델이 생성되었습니다.')
            
def arbitrary():
    Modeling()   

secondary_thread = threading.Thread(target=arbitrary)
    
button1=Button(frame, text = 'START',fg='White', bg="#212933" ,font="Bahnschrift 12", width=7, command=start_thread)
button1.place(x=870, y=235)
button2=Button(frame, text = 'EXIT', fg='White', bg="#212933" ,font="Bahnschrift 12",width=7,command=finish_prog)
button2.place(x=950, y=235)
progress_bar = ttk.Progressbar(frame, orient='horizontal', length=550, mode ='determinate')
progress_bar.place(x=290, y=240)

app.mainloop()